## Predictive Modeling of Food Deserts: Data Cleaning
### TDA Food Desert Project: Data Exploration & Alternative Data Sources
This notebook explores dataset relationships and identifies tract-level food environment data to supplement county-level FEA data.

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr

In [ ]:
# PART 1: EXPLORE DATASET RELATIONSHIPS
def explore_fea_county_coverage(fea_df, county_fips_col, merged_df):
    """
    Analyze how many tracts share the same county-level FEA data.
    This shows the data resolution problem.
    """
    print("="*70)
    print("FOOD ENVIRONMENT ATLAS COVERAGE ANALYSIS")
    print("="*70)
    
    # count tracts per county
    tracts_per_county = merged_df.groupby('COUNTYFIPS').size()
    
    print(f"\nTotal counties: {len(tracts_per_county)}")
    print(f"Total tracts: {len(merged_df)}")
    print(f"Avg tracts per county: {tracts_per_county.mean():.1f}")
    print(f"Median tracts per county: {tracts_per_county.median():.0f}")
    
    print("\nDistribution of tracts per county:")
    print(tracts_per_county.describe())
    
    # Counties with most tracts
    print("\nTop 10 counties by tract count:")
    top_counties = tracts_per_county.nlargest(10)
    for fips, count in top_counties.items():
        county_name = merged_df[merged_df['COUNTYFIPS']==fips]['County name'].iloc[0]
        print(f"  {fips} ({county_name}): {count} tracts")
    
    # Visualize
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(tracts_per_county, bins=30, edgecolor='black', alpha=0.7)
    plt.xlabel('Number of Tracts per County')
    plt.ylabel('Frequency')
    plt.title('Distribution of Tract Count per County')
    plt.axvline(tracts_per_county.mean(), color='red', linestyle='--', 
                label=f'Mean: {tracts_per_county.mean():.1f}')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.boxplot(tracts_per_county, vert=True)
    plt.ylabel('Tracts per County')
    plt.title('Tract Count Distribution (Boxplot)')
    
    plt.tight_layout()
    plt.show()
    
    return tracts_per_county


In [ ]:

def analyze_within_county_variation(merged_df):
    """
    Analyze variation of tract-level variables WITHIN counties.
    High variation suggests county-level FEA data may be insufficient.
    """
    print("\n" + "="*70)
    print("WITHIN-COUNTY VARIATION ANALYSIS")
    print("="*70)
    
    # Variables to analyze
    tract_vars = ['median_income', 'poverty_rate', 'population']
    tract_vars = [v for v in tract_vars if v in merged_df.columns]
    
    results = {}
    
    for var in tract_vars:
        # Calculate coefficient of variation within each county
        county_stats = merged_df.groupby('COUNTYFIPS')[var].agg([
            ('mean', 'mean'),
            ('std', 'std'),
            ('cv', lambda x: x.std() / x.mean() if x.mean() != 0 else 0),
            ('range', lambda x: x.max() - x.min())
        ])
        
        results[var] = county_stats
        
        print(f"\n{var.upper()}:")
        print(f"  Avg within-county CV: {county_stats['cv'].mean():.3f}")
        print(f"  Median within-county CV: {county_stats['cv'].median():.3f}")
        print(f"  Counties with high variation (CV > 0.5): {(county_stats['cv'] > 0.5).sum()}")
    
    # visualize
    fig, axes = plt.subplots(1, len(tract_vars), figsize=(5*len(tract_vars), 4))
    if len(tract_vars) == 1:
        axes = [axes]
    
    for idx, var in enumerate(tract_vars):
        axes[idx].hist(results[var]['cv'].dropna(), bins=30, edgecolor='black', alpha=0.7)
        axes[idx].set_xlabel('Coefficient of Variation')
        axes[idx].set_ylabel('Number of Counties')
        axes[idx].set_title(f'Within-County CV: {var}')
        axes[idx].axvline(0.5, color='red', linestyle='--', label='High variation threshold')
        axes[idx].legend()
    
    plt.tight_layout()
    plt.show()
    
    return results

In [ ]:
def identify_food_desert_indicators(merged_df):
    """
    Identify which FEA variables are most relevant for food desert analysis.
    """
    print("\n" + "="*70)
    print("FOOD DESERT INDICATOR IDENTIFICATION")
    print("="*70)
    
    # Key food access indicators in FEA (adjust based on your actual columns)
    food_access_keywords = [
        'snap', 'grocery', 'supermarket', 'store', 'access',
        'desert', 'distance', 'vehicle', 'low_income', 'poverty',
        'wic', 'assistance', 'market', 'fresh', 'healthy'
    ]
    
    # Find relevant columns
    relevant_cols = []
    for col in merged_df.columns:
        col_lower = col.lower()
        if any(keyword in col_lower for keyword in food_access_keywords):
            relevant_cols.append(col)
    
    print(f"\nFound {len(relevant_cols)} potential food access indicators:")
    
    # Calculate completeness and basic stats
    indicator_stats = []
    for col in relevant_cols:
        non_null = merged_df[col].notna().sum()
        pct_complete = (non_null / len(merged_df)) * 100
        
        try:
            numeric_vals = pd.to_numeric(merged_df[col], errors='coerce')
            mean_val = numeric_vals.mean()
            std_val = numeric_vals.std()
            indicator_stats.append({
                'Indicator': col,
                'Completeness': pct_complete,
                'Mean': mean_val,
                'Std': std_val
            })
        except:
            indicator_stats.append({
                'Indicator': col,
                'Completeness': pct_complete,
                'Mean': None,
                'Std': None
            })
    
    stats_df = pd.DataFrame(indicator_stats).sort_values('Completeness', ascending=False)
    print("\nTop 15 indicators by data completeness:")
    print(stats_df.head(15).to_string(index=False))
    
    return stats_df, relevant_cols


In [ ]:
# PART 2: ALTERNATIVE TRACT-LEVEL DATA SOURCES

def generate_tract_level_recommendations():
    """
    Provide comprehensive list of tract-level food environment data sources.
    """
    print("\n" + "="*70)
    print("ALTERNATIVE TRACT-LEVEL DATA SOURCES")
    print("="*70)
    
    sources = {
        "1. USDA Food Access Research Atlas": {
            "Description": "Tract-level food access indicators",
            "URL": "https://www.ers.usda.gov/data-products/food-access-research-atlas/",
            "Key Variables": [
                "- Low access tracts (0.5 or 1 mile from supermarket)",
                "- Low income & low access population",
                "- Vehicle access rates",
                "- SNAP authorized stores count",
                "- Distance to nearest supermarket"
            ],
            "Format": "CSV/Excel with tract FIPS codes",
            "Update Frequency": "Periodic (check for latest year)",
            "Integration": "Direct merge on GEOID"
        },
        
        "2. OpenStreetMap POI Data": {
            "Description": "Crowdsourced point-of-interest data for food outlets",
            "URL": "https://www.openstreetmap.org/",
            "Key Variables": [
                "- Grocery store locations (geocoded)",
                "- Convenience store locations",
                "- Restaurant locations by type",
                "- Farmer's market locations"
            ],
            "Format": "Requires processing (Overpass API or extracts)",
            "Update Frequency": "Real-time community updates",
            "Integration": "Spatial join to tracts; requires GIS processing",
            "Tools": "OSMnx Python library, QGIS"
        },
        
        "3. Google Places API": { # too computationally expensive for this project
            "Description": "Current food retail locations",
            "URL": "https://developers.google.com/maps/documentation/places/web-service",
            "Key Variables": [
                "- Supermarket locations",
                "- Grocery store locations",
                "- Restaurant locations",
                "- Ratings and business hours"
            ],
            "Format": "API (JSON)",
            "Update Frequency": "Real-time",
            "Integration": "Query by tract centroid + radius; aggregate counts",
            "Cost": "Free tier available; pay per request beyond limit"
        },
        
        "7. USDA SNAP Retailer Locator": {
            "Description": "Locations of SNAP-authorized retailers",
            "URL": "https://www.fns.usda.gov/snap/retailer-locator",
            "Key Variables": [
                "- SNAP store locations by type",
                "- Store authorization status"
            ],
            "Format": "Searchable database; may require scraping or FOIA request",
            "Update Frequency": "Regular updates",
            "Integration": "Geocode addresses, spatial join to tracts"
        },
        
        "8. ACS Table B22001": {
            "Description": "Receipt of Food Stamps/SNAP by tract",
            "URL": "https://data.census.gov/",
            "Key Variables": [
                "- Households receiving SNAP benefits",
                "- SNAP participation rate"
            ],
            "Format": "Census API or bulk download",
            "Update Frequency": "Annual (5-year estimates)",
            "Integration": "Already in ACS; look for table B22001"
        }
    }
    
    for source_name, details in sources.items():
        print(f"\n{source_name}")
        print("-" * 70)
        for key, value in details.items():
            if isinstance(value, list):
                print(f"{key}:")
                for item in value:
                    print(f"  {item}")
            else:
                print(f"{key}: {value}")
    
    return sources



In [ ]:

def create_synthetic_tract_features(merged_df):
    """
    Create tract-level proxies when direct food access data unavailable.
    Uses existing socioeconomic and demographic data.
    """
    print("\n" + "="*70)
    print("SYNTHETIC TRACT-LEVEL FEATURE ENGINEERING")
    print("="*70)
    
    df = merged_df.copy()
    
    # 1. urbanicity proxy (based on population density)
    # 2029 data contains urban column denoted by 1- urban area, 0 - not urban
    if 'density' in df.columns:
        df['urbanicity_score'] = pd.cut(
            df['density'],
            bins=[0, 1000, 4000, 10000, np.inf],
            labels=['Rural', 'Suburban', 'Urban', 'Dense_Urban']
        )
        print("✓ Created urbanicity_score (based on population density)")
    
    # 2. Economic disadvantage
    if 'poverty_rate' in df.columns and 'median_income' in df.columns:
        # Normalize both to 0-1 scale
        poverty_norm = (df['poverty_rate'] - df['poverty_rate'].min()) / \
                    (df['poverty_rate'].max() - df['poverty_rate'].min())
        income_norm = 1 - ((df['median_income'] - df['median_income'].min()) / \
                        (df['median_income'].max() - df['median_income'].min()))
        
        df['economic_disadvantage'] = (poverty_norm + income_norm) / 2
        print("✓ Created economic_disadvantage (poverty + low income composite)")
    
    # 3. Food desert risk score (composite of available factors)
    risk_components = []
    
    if 'poverty_rate' in df.columns:
        risk_components.append('poverty_rate')
    
    if 'vehicle_access' in df.columns:  # if available from ACS
        risk_components.append('vehicle_access')
        # vehicle access is important as it determines things like travel times to stores which affects food access
    
    if 'density' in df.columns:
        # Low density increases risk in rural areas - may have longer travel time to stores 
        df['low_density_risk'] = 1 / (df['density'] + 1)  # inverse relationship
        risk_components.append('low_density_risk')
    
    if len(risk_components) > 0:
        # normalize and average
        risk_normalized = df[risk_components].apply(
            lambda x: (x - x.min()) / (x.max() - x.min()) if x.std() > 0 else 0
        )
        df['food_desert_risk_score'] = risk_normalized.mean(axis=1)
        print(f"✓ Created food_desert_risk_score (using {len(risk_components)} components)")
    
    # 4. neighborhood disadvantage index - combinign fators for each neighborhodd that will dtermine how at risk the neighborhood is to food insecurity
    socioeconomic_vars = ['poverty_rate', 'median_income', 'unemployment_rate']
    available_vars = [v for v in socioeconomic_vars if v in df.columns]
    
    if len(available_vars) >= 2:
        from sklearn.preprocessing import StandardScaler
        from sklearn.decomposition import PCA
        
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(df[available_vars].fillna(df[available_vars].median()))
        
        pca = PCA(n_components=1)
        df['neighborhood_disadvantage_index'] = pca.fit_transform(scaled_data)
        
        print(f"✓ Created neighborhood_disadvantage_index (PCA of {len(available_vars)} variables)")
        print(f"  Explained variance: {pca.explained_variance_ratio_[0]:.2%}")
    
    new_features = [col for col in df.columns if col not in merged_df.columns]
    print(f"\nTotal new features created: {len(new_features)}")
    print("New features:", new_features)
    
    return df



In [ ]:
# PART 3: DATA INTEGRATION RECOMMENDATIONS

def recommend_integration_strategy(merged_df):
    """
    Provide specific recommendations for data integration strategy.
    """
    print("\n" + "="*70)
    print("RECOMMENDED DATA INTEGRATION STRATEGY")
    print("="*70)
    
    print("\nIMMEDIATE ACTIONS (Use existing data):")
    print("-" * 70)
    print("1. Extract all available food access variables from FEA")
    print("   - Even though county-level, provides baseline")
    print("   - Key variables: SNAP stores, grocery stores, distance to stores")
    print()
    print("2. Enhance with ACS tract-level data:")
    print("   - B22001: SNAP/Food Stamp receipt")
    print("   - B08201: Household vehicle availability")
    print("   - B19058: Public assistance income")
    print("   - B25014: Occupants per room (crowding)")
    print()
    print("3. Create synthetic features from existing data")
    print("   - Population density as urbanicity proxy")
    print("   - Economic disadvantage composite")
    print("   - Food desert risk score")
    
    print("\n SHORT-TERM ENHANCEMENTS (1-2 weeks):")
    print("-" * 70)
    print("1. Download USDA Food Access Research Atlas")
    print("   - Direct tract-level food access indicators")
    print("   - FREE and well-documented")
    print("   - High quality government data")
    print()
    print("2. Scrape OpenStreetMap POI data")
    print("   - Use OSMnx or Overpass API")
    print("   - Count grocery stores per tract")
    print("   - Calculate distance to nearest store from tract centroid")
    print()
    print("3. Query Google Places API for current store locations") # deciding against due to computationally heavy + cost
    print("   - Supplement OSM with more recent data")
    print("   - Focus on grocery/supermarket categories")
    
    print("\n LONG-TERM ENHANCEMENTS (If resources available):")
    print("-" * 70)
    print("1. Access SafeGraph Core Places (free academic license)")
    print("   - High-quality commercial POI data")
    print("   - Includes visit patterns and demographics")
    print()
    print("2. Integrate CDC PLACES health outcome data")
    print("   - Validate food desert predictions")
    print("   - Obesity and diabetes as outcome variables")
    print()
    print("3. Consider PolicyMap subscription")
    print("   - Comprehensive food environment metrics")
    print("   - Academic discounts available")
    
    print("\n  DATA QUALITY CONSIDERATIONS:")
    print("-" * 70)
    print("• County-level FEA limitations:")
    print("  - All tracts in same county get identical values")
    print("  - Reduces model's ability to distinguish within-county variation")
    print("  - May artificially inflate homology features in urban counties")
    print()
    print("• Recommended approach:")
    print("  - Use FEA as baseline/control variables")
    print("  - Prioritize tract-level food access data for main analysis")
    print("  - Create distance-based features (tract centroid to stores)")
    
    return None



In [ ]:
# main execution
if __name__ == "__main__":
    # Load your merged dataset
    merged_df = pd.read_csv('merged_data_cleaned.csv')
    
    print("STEP 1: Analyzing current data limitations...")
    tracts_per_county = explore_fea_county_coverage(None, None, merged_df)
    
    print("\nSTEP 2: Examining within-county variation...")
    variation_results = analyze_within_county_variation(merged_df)
    
    print("\nSTEP 3: Identifying existing food desert indicators...")
    indicator_stats, relevant_cols = identify_food_desert_indicators(merged_df)
    
    print("\nSTEP 4: Documenting alternative data sources...")
    sources = generate_tract_level_recommendations()
    
    print("\nSTEP 5: Creating synthetic features from existing data...")
    enhanced_df = create_synthetic_tract_features(merged_df)
    
    print("\nSTEP 6: Providing integration recommendations...")
    recommend_integration_strategy(merged_df)
    
    # Save enhanced dataset
    enhanced_df.to_csv('merged_data_enhanced.csv', index=False)
    print("\n✓ Saved enhanced dataset with synthetic features")